# SiLA Device Discovery

[SiLA](https://sila-standard.com/) is a communication standard used by many lab instruments. PyLabRobot can discover both SiLA 1 and SiLA 2 devices on the local network:

- **SiLA 1** devices are found via NetBIOS broadcast + SOAP `GetDeviceIdentification`
- **SiLA 2** devices advertise via mDNS (`_sila._tcp.local.`)

## Requirements

SiLA 1 discovery works out of the box (no extra dependencies). SiLA 1 devices typically use link-local addresses (169.254.x.x).

For SiLA 2 discovery (mDNS):

```bash
pip install zeroconf
```

Your computer must be on the same network (or VLAN) as the instruments you want to discover.

## Usage

In [ ]:
from pylabrobot.io.sila.discovery import discover

devices = await discover(timeout=5.0)
print(f"Found {len(devices)} device(s)")
for device in devices:
  print(device)

`discover()` runs SiLA 1 (NetBIOS + SOAP) and SiLA 2 (mDNS) probes in parallel, listening for `timeout` seconds. It returns a list of `SiLADevice` objects:

| Attribute          | Type            | Description                           |
|--------------------|-----------------|---------------------------------------|
| `host`             | `str`           | IP address (e.g. `192.168.1.42`)      |
| `port`             | `int`           | Service port (e.g. `8080` or `8091`)  |
| `name`             | `str`           | Device name                           |
| `serial_number`    | `Optional[str]` | Serial number (SiLA 1 only, else `None`) |
| `firmware_version` | `Optional[str]` | Firmware version (SiLA 1 only, else `None`) |
| `sila_version`     | `int`           | `1` or `2`                            |

## Example: connecting to the first device found

You can use the discovered host and port directly when creating a backend:

```python
from pylabrobot.io.sila.discovery import discover

devices = await discover()
assert len(devices) > 0, "No SiLA devices found"
device = devices[0]

# Pass device.host and device.port to your backend
backend = SomeBackend(host=device.host, port=device.port)
```

## Command-line tool

You can also discover devices from the terminal:

```bash
python -m pylabrobot.io.sila.discovery
```

Use `-t` to change the timeout (default 5 seconds):

```bash
python -m pylabrobot.io.sila.discovery -t 10
```

For SiLA 1 devices on a specific link-local interface:

```bash
python -m pylabrobot.io.sila.discovery --interface 169.254.183.87
```

Output is tab-delimited, one device per line:

```
169.254.151.99	8080	ODTC_1A3C93	SiLA 1	S/N:12345
192.168.1.42	8091	ImageXpress-Pico	SiLA 2
```